In [1]:
import numpy as np
import pandas as pd
import volprice as vl
from tqdm import tqdm_notebook
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from itertools import product
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
import logging
import inspect

In [2]:
%matplotlib inline
mpl.style.use('seaborn-talk')

In [3]:
logging.basicConfig(filename='convergence.tmp.log', level=logging.INFO)
logging.captureWarnings(True)

## Setup the True Parameters /  Simulation Parameters

In [4]:
true_params = {'equity_price': 0.50, 
               'vol_price': -7,
               'phi':-.17,
               'scale': .1,
               'delta': 1.22,
               'rho': .8,
              }
time_dim = 3000

## Setup the linking functions.

In [5]:
total_true_params = true_params.copy()
total_true_params.update(
    {'gamma': vl.gamma(rho=true_params['rho'], scale=true_params['scale'], delta=true_params['delta'], 
                       phi=true_params['phi'], pi=true_params['vol_price'], theta=true_params['equity_price']),
     'beta':vl.beta(true_params['rho'], true_params['scale'], true_params['phi'], pi=true_params['vol_price'], 
                    theta=true_params['equity_price']),
     'psi':vl.psi(true_params['rho'], true_params['scale'], true_params['phi'], 
                  theta=true_params['equity_price']), 
     'phi_squared': true_params['phi']**2})

TypeError: _lambdifygenerated() got multiple values for argument 'pi'

## We start by examining the distributions of simulated data

In [ ]:
data = vl.simulate_data(time_dim=time_dim, **true_params)

In [ ]:
fig1, ax1 = mpl.pyplot.subplots()
ax1.plot(data.rtn, label='Return')
ax1.plot(data.vol, label='Volatility')
ax1.legend()


In [ ]:
sns.jointplot(x='vol', y='rtn', data=data, kind='reg')

In [ ]:
data.describe()

In [ ]:
init_constants = vl.compute_init_constants(data.vol)

In [ ]:
num_vals = 50
cvals = np.linspace(0, 1,num_vals)
deltavals = np.linspace(0,1.5,num_vals)

In [ ]:
plot_data = pd.DataFrame(np.asarray([(scale, intercept, vl.vol_moments(data.vol, true_params['rho'],
                                                                       scale, intercept).apply(
    np.square).sum().mean())
     for scale, intercept in product(cvals, deltavals)]), columns=['c', 'delta', 'z'])
reshaped_x = plot_data.c.values.reshape((num_vals,num_vals))
reshaped_y = plot_data.delta.values.reshape((num_vals,num_vals))
reshaped_z = plot_data.z.values.reshape((num_vals,num_vals))

In [ ]:
fig = mpl.pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(reshaped_x, reshaped_y, reshaped_z , cmap=cm.coolwarm, alpha=.5)

ax.set_ylabel('delta')
ax.set_zlabel('squared_errors')
ax.set_xlabel('scale')

In [ ]:
vl.vol_moments_grad(data.vol, 1.22, .8, .1)

In [ ]:
params_1st_stage, cov_1st_stage = vl.compute_vol_gmm(data.vol, init_constants=init_constants)

In [ ]:
params_1st_stage

In [ ]:
cov_1st_stage

In [ ]:
reduced_form_est = vl.create_est_table(estimates=params_1st_stage, truth=true_params, cov=cov_1st_stage)

In [ ]:
reduced_form_est

# Step 2

In [ ]:
step2_est, step2_cov = vl.compute_step2(data)

In [ ]:
params_1st_stage.update(step2_est)
cov_stage_1a = cov_1st_stage.copy().merge(step2_cov, left_index=True, right_index=True, how='outer').fillna(0)

In [ ]:
vl.create_est_table(estimates=params_1st_stage, truth=total_true_params, cov=cov_stage_1a)


## I now setup the second-stage derivation

We sometimes get invalid values inside the logarithm for non-optimal values. This would induce a complicated nonlinear constraint for the valid values of the volatilty risk price, but they do not bind, and so we don't bother imposing them.

In [ ]:
params_2nd_stage = vl.est_2nd_stage(reduced_form_params=params_1st_stage, reduced_form_cov=cov_stage_1a)

In [ ]:
params_2nd_stage

## I now compute the covariance matrix for this 2nd stage regression

I first compute the bread for the sandwich

In [ ]:
cov_2nd_stage = vl.compute_2nd_stage_cov(params2=params_2nd_stage, cov1=cov_stage_1a, time_dim=1)

In [ ]:
vl.create_est_table(estimates=params_2nd_stage, truth=total_true_params, cov=cov_2nd_stage)

In [ ]:
vl.cov_to_corr(cov_2nd_stage)